# 🎙️ XTTS-v2 Deepfake Speech Analysis

LibriSpeech 데이터셋을 사용하여 XTTS-v2로 음성을 생성하고, Real vs Generated 음성을 분석합니다.

### 분석 내용
1. **Gradient Field 분석**: Mel Spectrogram의 Gradient를 t-SNE, UMAP으로 시각화
2. **Whisper Encoder 분석**: Whisper 모델의 encoder features를 UMAP으로 시각화


In [1]:
# 라이브러리 Import
import pandas as pd
import os
import time
import soundfile as sf
import matplotlib.pyplot as plt
import torch
import numpy as np
import librosa
import umap
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from transformers import WhisperProcessor, WhisperModel
import torchaudio

# PyTorch 2.6+ weights_only 이슈 해결
import torch.serialization
torch.serialization.add_safe_globals([dict])
# 또는 전역적으로 weights_only=False 사용하도록 설정
import functools
_original_torch_load = torch.load
torch.load = functools.partial(_original_torch_load, weights_only=False)

from TTS.api import TTS


/mnt/fr20tb/jihwan/.conda/envs/tts/lib/python3.10/site-packages/umap/__init__.py:35: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/mnt/fr20tb/jihwan/.conda/envs/tts/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 경로 설정
N_SAMPLES = 1000
LIBRISPEECH_ROOT = "./my_librispeech/LibriSpeech"
LIBRISPEECH_SUBSET = "test-clean"
OUTPUT_DIR = Path(f"generated_results_XTTS_v2_{N_SAMPLES}").resolve()
PROMPT_WAV_DIR = Path("prompt_wav_files").resolve()
FIGURE_DIR = Path(f"analysis_figures_xtts_v2_{N_SAMPLES}").resolve()
CUDA_DEVICE = "0"

# 폴더 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PROMPT_WAV_DIR, exist_ok=True)
os.makedirs(FIGURE_DIR, exist_ok=True)

print(f"📁 OUTPUT_DIR: {OUTPUT_DIR}")
print(f"📁 PROMPT_WAV_DIR: {PROMPT_WAV_DIR}")
print(f"📁 FIGURE_DIR: {FIGURE_DIR}")


📁 OUTPUT_DIR: /mnt/ddn/jihwan/projects/code/Deepfake-speech/generated_results_XTTS_v2_Random
📁 PROMPT_WAV_DIR: /mnt/ddn/jihwan/projects/code/Deepfake-speech/prompt_wav_files
📁 FIGURE_DIR: /mnt/ddn/jihwan/projects/code/Deepfake-speech/analysis_figures_xtts_v2


In [3]:
# 데이터프레임 구축 및 샘플링
from libri_dataframe import build_librispeech_dataframe

print("=== 데이터프레임 구축 및 샘플링 ===")
dataframe = build_librispeech_dataframe(
    librispeech_root=LIBRISPEECH_ROOT,
    subset=LIBRISPEECH_SUBSET,
)

# [핵심] 다양한 화자가 섞이도록 랜덤 샘플링 (100개)
dataframe_sampled = dataframe.sample(n=100, random_state=42).reset_index(drop=True)
print(f"✅ 샘플링 완료: 100개 파일 (화자 수: {len(dataframe_sampled['speaker_id'].unique())}명)")
dataframe_sampled.head()


=== 데이터프레임 구축 및 샘플링 ===
Loaded 2620 utterances from my_librispeech/LibriSpeech/test-clean
  - Speakers: 40
  - Chapters: 87
✅ 샘플링 완료: 100개 파일 (화자 수: 37명)


,speaker_id,chapter_id,utterance_id,transcript,audio_path
0,7729,102255,7729-102255-0008,ALL THE TERRITORIAL DIGNITARIES WERE PRESENT G...,my_librispeech/LibriSpeech/test-clean/7729/102...
1,1221,135766,1221-135766-0004,THIS OUTWARD MUTABILITY INDICATED AND DID NOT ...,my_librispeech/LibriSpeech/test-clean/1221/135...
2,4507,16021,4507-16021-0032,HE MUST DESCEND WITH HIS HEART FULL OF CHARITY...,my_librispeech/LibriSpeech/test-clean/4507/160...
3,5105,28241,5105-28241-0019,NOTHING WAS TO BE DONE BUT TO PUT ABOUT AND RE...,my_librispeech/LibriSpeech/test-clean/5105/282...
4,8463,294825,8463-294825-0001,THIS REALITY BEGINS TO EXPLAIN THE DARK POWER ...,my_librispeech/LibriSpeech/test-clean/8463/294...


## 2. 오디오 생성 (XTTS-v2)


In [4]:
# 헬퍼 함수들
def resolve_path(path_str): 
    return str(Path(path_str).resolve())

def normalize_text(text):
    text = text.lower().strip()
    if not text.endswith(('.', '!', '?', ',')): 
        text += '.'
    return text

def convert_to_wav(input_path, output_path):
    try:
        data, samplerate = sf.read(input_path)
        sf.write(output_path, data, samplerate)
    except: 
        pass


In [5]:
# XTTS-v2 모델 초기화
print("=== XTTS-v2 모델 로딩 ===")
os.environ["CUDA_VISIBLE_DEVICES"] = CUDA_DEVICE
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)
print("✅ XTTS-v2 모델 로드 완료")


=== XTTS-v2 모델 로딩 ===
 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.


/mnt/fr20tb/jihwan/.conda/envs/tts/lib/python3.10/site-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > Using model: xtts
✅ XTTS-v2 모델 로드 완료


In [6]:
# 프롬프트 준비 및 생성 리스트업
input_data = []

for idx, row in enumerate(dataframe_sampled.to_dict('records')):
    spk_id = row['speaker_id']
    output_path = OUTPUT_DIR / f"gen_random_{idx:03d}_{spk_id}.wav"
    
    # 프롬프트용 WAV 변환 (없으면 생성)
    prompt_wav = PROMPT_WAV_DIR / f"prompt_{spk_id}.wav"
    if not prompt_wav.exists():
        convert_to_wav(resolve_path(row['audio_path']), str(prompt_wav))

    input_data.append({
        "idx": idx,
        "original_path": resolve_path(row['audio_path']),
        "output_path": str(output_path),
        "text": row['transcript'],
        "prompt_wav": str(prompt_wav)
    })

print(f"✅ 준비 완료: {len(input_data)}개 샘플")


✅ 준비 완료: 100개 샘플


In [7]:
# 오디오 생성 (XTTS-v2)
print("=== 오디오 생성 확인 (XTTS-v2) ===")
for item in input_data:
    if not os.path.exists(item['output_path']):
        # 파일이 없을 때만 생성 수행
        print(f"Generating {Path(item['output_path']).name}...")
        try:
            tts.tts_to_file(
                text=normalize_text(item['text']),
                file_path=item['output_path'],
                speaker_wav=item['prompt_wav'],
                language="en"
            )
        except Exception as e:
            print(f"Error generating {item['output_path']}: {e}")

print("✅ 오디오 생성 완료!")


=== 오디오 생성 확인 (XTTS-v2) ===
Generating gen_random_000_7729.wav...
 > Text splitted to sentences.
['all the territorial dignitaries were present governor shannon presided john calhoun the surveyor general made the principal speech a denunciation of the abolitionists supporting the topeka movement chief justice lecompte dignified the occasion with approving remarks.']
[!] Warning: The text length exceeds the character limit of 250 for language 'en', this might cause truncated audio.
 > Processing time: 11.043159008026123
 > Real-time factor: 0.6054363491242393
Generating gen_random_001_1221.wav...
 > Text splitted to sentences.
['this outward mutability indicated and did not more than fairly express the various properties of her inner life.']
 > Processing time: 3.7742600440979004
 > Real-time factor: 0.48228114263072963
Generating gen_random_002_4507.wav...
 > Text splitted to sentences.
['he must descend with his heart full of charity and severity at the same time as a brother and as a

KeyboardInterrupt: 

## 3. 시각화 함수 정의


In [ ]:
def plot_and_save(X_embedded, y, filenames, labels, title, filename_suffix):
    """Real vs Generated 시각화 (화살표 포함)"""
    plt.figure(figsize=(14, 11))
    
    target_groups = ["Original (Real)", "Gen (XTTS-v2)"]
    colors = ['blue', 'green']
    markers = ['o', '^']

    # 점 찍기
    for i, group_name in enumerate(target_groups):
        mask = (y == i)
        plt.scatter(
            X_embedded[mask, 0], X_embedded[mask, 1],
            c=colors[i], label=group_name, marker=markers[i],
            s=100, alpha=0.7, edgecolors='white', zorder=2
        )

    # 화살표 그리기 (Real → Gen)
    unique_names = sorted(list(set(filenames)))
    for name in unique_names:
        indices_real = [i for i, (f, l) in enumerate(zip(filenames, labels)) if f == name and l == 0]
        indices_gen = [i for i, (f, l) in enumerate(zip(filenames, labels)) if f == name and l == 1]
        
        if indices_real and indices_gen:
            start = X_embedded[indices_real[0]]
            end = X_embedded[indices_gen[0]]
            plt.annotate("", xy=end, xytext=start,
                         arrowprops=dict(arrowstyle="-|>", color='gray', alpha=0.3, linewidth=1),
                         zorder=1)

    plt.title(title, fontsize=16, fontweight='bold')
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3, linestyle='--')
    plt.tight_layout()
    
    save_path = FIGURE_DIR / f"{filename_suffix}.png"
    plt.savefig(save_path, dpi=300)
    print(f"✅ 저장 완료: {save_path}")
    plt.show()
    plt.close()


## 4. 분석 1: Gradient Field (t-SNE & UMAP)

Mel Spectrogram에서 Gradient를 추출하여 Real vs Generated 음성을 비교합니다.


In [ ]:
def get_gradient_features(audio_path):
    """오디오에서 Mel Spectrogram Gradient 특징 추출"""
    try:
        waveform, sr = torchaudio.load(audio_path)
        # Mel Spectrogram
        mel = torchaudio.transforms.MelSpectrogram(
            sample_rate=sr, n_fft=1024, hop_length=256, n_mels=80, normalized=True
        )(waveform)
        mel_db = torchaudio.functional.amplitude_to_DB(mel, multiplier=10., amin=1e-10, db_multiplier=1.)
        
        # Resize to fixed width (256)
        target_width = 256
        n_mels, width = mel_db.squeeze(0).shape
        if width > target_width:
            mel_db = mel_db[:, :, :target_width]
        else:
            pad = target_width - width
            mel_db = torch.nn.functional.pad(mel_db, (0, pad))
            
        # Gradient Calculation
        mel_np = mel_db.squeeze(0).numpy()
        grad_y, grad_x = np.gradient(mel_np)
        grad_mag = np.sqrt(grad_x**2 + grad_y**2)
        return grad_mag.flatten()  # Flatten vector
    except:
        return None


In [ ]:
# Gradient Feature 추출
print("[Analysis 1] Gradient Field Feature Extraction...")

features = []
labels = []
filenames = []

for item in input_data:
    # Real
    f = get_gradient_features(item['original_path'])
    if f is not None:
        features.append(f)
        labels.append(0)
        filenames.append(f"s_{item['idx']}")
    # Gen
    if os.path.exists(item['output_path']):
        f = get_gradient_features(item['output_path'])
        if f is not None:
            features.append(f)
            labels.append(1)
            filenames.append(f"s_{item['idx']}")

X_grad = np.array(features)
y_grad = np.array(labels)
scaler = StandardScaler()
X_grad_scaled = scaler.fit_transform(X_grad)

print(f"✅ Gradient Data Shape: {X_grad.shape}")


In [ ]:
# Gradient t-SNE
print("Running Gradient t-SNE...")
tsne = TSNE(n_components=2, perplexity=30, random_state=42, init='pca', learning_rate='auto')
X_tsne = tsne.fit_transform(X_grad_scaled)
plot_and_save(X_tsne, y_grad, filenames, labels,
              "t-SNE (Gradient Field): Real vs Gen Flow", 
              "Gradient_Field_tSNE")


In [ ]:
# Gradient UMAP
print("Running Gradient UMAP...")
reducer = umap.UMAP(n_neighbors=15, min_dist=0.3, random_state=42)
X_umap_grad = reducer.fit_transform(X_grad_scaled)
plot_and_save(X_umap_grad, y_grad, filenames, labels,
              "UMAP (Gradient Field): Real vs Gen Flow", 
              "Gradient_Field_UMAP")


## 5. 분석 2: Whisper Encoder (UMAP)

Whisper 모델의 encoder features를 사용하여 Real vs Generated 음성을 비교합니다.


In [ ]:
# Whisper 모델 로드
print("[Analysis 2] Whisper Feature Extraction...")

device = "cuda" if torch.cuda.is_available() else "cpu"
whisper_model = WhisperModel.from_pretrained("openai/whisper-base").to(device)
whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-base")
print(f"✅ Whisper 모델 로드 완료 (device: {device})")


In [ ]:
def get_whisper_features(audio_path):
    """Whisper encoder로 오디오 특징 추출"""
    try:
        audio, _ = librosa.load(str(audio_path), sr=16000, mono=True)
        if len(audio) > 30*16000: 
            audio = audio[:30*16000]  # 30초 제한
        
        inputs = whisper_processor(audio, sampling_rate=16000, return_tensors="pt")
        input_features = inputs.input_features.to(device)
        
        with torch.no_grad():
            outputs = whisper_model.encoder(input_features)
        
        # Mean Pooling
        return outputs.last_hidden_state.mean(dim=1).squeeze(0).cpu().numpy()
    except Exception as e:
        print(e)
        return None


In [ ]:
# Whisper Feature 추출
features_w = []
labels_w = []
filenames_w = []

for item in input_data:
    # Real
    f = get_whisper_features(item['original_path'])
    if f is not None:
        features_w.append(f)
        labels_w.append(0)
        filenames_w.append(f"s_{item['idx']}")
    # Gen
    if os.path.exists(item['output_path']):
        f = get_whisper_features(item['output_path'])
        if f is not None:
            features_w.append(f)
            labels_w.append(1)
            filenames_w.append(f"s_{item['idx']}")

X_whisper = np.array(features_w)
y_whisper = np.array(labels_w)
X_whisper_scaled = scaler.fit_transform(X_whisper)

print(f"✅ Whisper Data Shape: {X_whisper.shape}")


In [ ]:
# Whisper UMAP
print("Running Whisper UMAP...")
reducer_w = umap.UMAP(n_neighbors=15, min_dist=0.3, random_state=42)
X_umap_whisper = reducer_w.fit_transform(X_whisper_scaled)
plot_and_save(X_umap_whisper, y_whisper, filenames_w, labels_w,
              "UMAP (Whisper Features): Real vs Gen Flow", 
              "Whisper_Encoder_UMAP")


In [ ]:
print("\n" + "="*50)
print("🎉 모든 분석 완료!")
print(f"📁 결과 저장 폴더: {FIGURE_DIR}")
print("="*50)
